In [1]:
import google.generativeai as genai
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv
import os


SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
你將會得知病人的ICD-10代碼跟病例並回推為什麼病人的ICD-10是如此：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結病人的ICD-10編碼集合。  
</reasoning>  
<answer>  
僅輸出正確的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""

EXAMPLE_QUESTION = """
There is a patient with the following ICD-10 codes: C3432,C779,D684,E8352,E861,F200,G8911,I214,I82431,J439,J45909,J9601,J9602,M5430,N170,R578,R64,S0012XD,S0083XD,T4275XA,W1830XS,Y92122,Y92129,Z515,Z5329,Z681,Z781,Z8781,Z87891,Z9181,Z9981.

The medical record for the Patient is as follows:
Sex:   F\n \nService: MEDICINE\n \nAllergies: \nThorazine / Stelazine / Haloperidol / Methylphenidate / peanut\n \nAttending: ___.\n \nChief Complaint:\nhypoxia\n \nMajor Surgical or Invasive Procedure:\nnone\n \nHistory of Present Illness:\n___ yo F with end stage COPD (on 2L home O2), psychosis, recently \ndiscovered LLL mass highly concerning for primary lung \nmalignancy, on hospice but full code, sent over from her rehab \nwith hypoxia. \n\nHPI limited as patient unable to participate. Per review of OMR \nnotes at baseline she is conversant though at times \ntangential/repetitive, ambulates w/o assistance, and is on 2L
Please help me, using the medical record above, infer why each of this patient's ICD-10 codes was assigned. Please respond in the specified format.
"""
EXAMPLE_REASONING = """Step-by-step reasoning:
Respiratory System and Malignancy (Primary Issue):
C3432 (Malignant neoplasm of lower lobe, left bronchus or lung): Directly corresponds to the "LLL mass highly concerning for primary lung malignancy."
C779 (Secondary malignant neoplasm of lymph node): Likely indicates metastatic spread, consistent with the "end stage" description of the malignancy.
J439 (Emphysema, unspecified): Corresponds to "end stage COPD."
J9601 (Acute respiratory failure with hypoxia) & J9602 (Acute respiratory failure with hypercapnia): Corresponds to the chief complaint of "hypoxia" and the respiratory distress associated with COPD.
J45909 (Unspecified asthma): Often comorbid with COPD (Asthma-COPD overlap), likely present in the full history.
Z515 (Encounter for palliative care): Matches the status "on hospice."
Z9981 (Dependence on supplemental oxygen): Matches "on 2L home O2."

Psychiatric Status:
F200 (Paranoid schizophrenia): Corresponds to the history of "psychosis" and the allergy list containing multiple antipsychotics (Thorazine, Stelazine, Haloperidol), suggesting a long-term psychiatric diagnosis.
Trauma and Fall (Context from "sent over from her rehab"):
S0012XD (Contusion of left eyelid and periocular area) & S0083XD (Contusion of other part of head): Indicates the patient suffered a fall or trauma recently. "XD" denotes a subsequent encounter (healing phase).
W1830XS (Fall on same level): The external cause code for the fall.
Y92122 & Y92129 (Place of occurrence: Nursing home/Rehab): Corresponds to being "sent over from her rehab."
G8911 (Acute pain due to trauma): Pain associated with the fall injuries.
Z9181 (History of falling): Documents the risk factor.

Acute Comorbidities and Complications (Inpatient Findings):
I214 (Non-ST elevation (NSTEMI) myocardial infarction): Acute heart strain likely triggered by the severe hypoxia.
I82431 (Acute embolism and thrombosis of right popliteal vein): Deep Vein Thrombosis (DVT), a high risk for immobile cancer patients.
N170 (Acute kidney failure with tubular necrosis): Likely due to dehydration or hypoperfusion (shock).
D684 (Acquired coagulation factor deficiency) & E8352 (Hypercalcemia): Metabolic and blood disorders often associated with malignancy (paraneoplastic syndromes).
E861 (Volume depletion/Dehydration): Matches the description "unable to participate," suggesting poor intake.

Other History and Status:
T4275XA (Adverse effect of antiepileptic and sedative-hypnotic drugs): Suggests a medication issue, possibly contributing to the fall or mental status change.
Z87891 (Personal history of nicotine dependence): Standard coding for COPD/Lung Cancer patients.
Z681 (BMI < 19.9) & R64 (Cachexia): Reflects the physical wasting associated with end-stage cancer.
Summary: The codes depict an elderly female with end-stage lung cancer and COPD on hospice, who was transferred from rehab due to hypoxia and acute respiratory failure, complicated by a recent fall, NSTEMI, and acute kidney injury.
"""

EXAMPLE_ANSWER = "C3432,C779,D684,E8352,E861,F200,G8911,I214,I82431,J439,J45909,J9601,J9602,M5430,N170,R578,R64,S0012XD,S0083XD,T4275XA,W1830XS,Y92122,Y92129,Z515,Z5329,Z681,Z781,Z8781,Z87891,Z9181,Z9981"

def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)

def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    return f1

c:\Users\Yuan\Desktop\YUAN\實習\國泰實習\醫療代碼分類\icd_google_api_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


START_INDEX = 0     # 起始索引(從0開始)
END_INDEX = 170     # 結束索引(不包含此索引)

genai.configure(api_key=" ")

print(f"✅ 正在設定模型: gemini-2.5-flash ...")
print(f"📊 處理區間: 第 {START_INDEX} 至 第 {END_INDEX-1} 筆 (共 {END_INDEX-START_INDEX} 筆)\n")

# generation_config = {
#     "temperature": 0,
#     "top_p": 1,
#     "top_k": 1,
#     "max_output_tokens": 8192,
# }

# 建立模型實例
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SYSTEM_PROMPT
)

print("✅ 模型設定完畢。")

# --- 5. 載入並處理資料 ---
print("\n📂 正在載入 note_icd_data_with_ans_with_ans_with_ans.jsonl ...")
chat_style_data = []

try:
    with open("note_icd_data_with_ans.jsonl", "r", encoding="utf-8") as f:
        all_lines = f.readlines()
        total_records = len(all_lines)
        
        print(f"📊 檔案總共有 {total_records} 筆資料")
        
        # 驗證索引範圍
        if START_INDEX < 0 or END_INDEX > total_records:
            print(f"❌ 錯誤：索引範圍超出資料範圍 (0-{total_records})")
            exit()
        
        if START_INDEX >= END_INDEX:
            print(f"❌ 錯誤：起始索引必須小於結束索引")
            exit()
        
        # 只讀取指定區間的資料
        lines = all_lines[START_INDEX:END_INDEX]
        print(f"✅ 已選取第 {START_INDEX} 至 第 {END_INDEX-1} 筆，共 {len(lines)} 筆\n")

        for line in tqdm(lines, desc="[Gemini] 載入並預處理資料"):
            item = json.loads(line)
            ground_truth_answer = extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })

except FileNotFoundError:
    print("❌ 錯誤：找不到 'note_icd_data_with_ans.jsonl' 檔案。")
    exit()

print(f"✅ 資料載入完畢，共 {len(chat_style_data)} 筆。\n")

✅ 正在設定模型: gemini-2.5-flash ...
📊 處理區間: 第 0 至 第 169 筆 (共 170 筆)

✅ 模型設定完畢。

📂 正在載入 note_icd_data_with_ans_with_ans_with_ans.jsonl ...
📊 檔案總共有 3852 筆資料
✅ 已選取第 0 至 第 169 筆，共 170 筆



[Gemini] 載入並預處理資料: 100%|██████████| 170/170 [00:00<00:00, 12760.28it/s]

✅ 資料載入完畢，共 170 筆。



In [3]:
# --- 6. 執行推論並計算指標 ---
all_predictions = []
all_ground_truths = []
all_f1_scores = []
all_completion_lengths = []

# 生成帶區間標記的 CSV 檔名
csv_filename = f"gemini_flash_gt_reasonning_{START_INDEX}_{END_INDEX}.csv"

# 檢查檔案是否已存在
file_exists = os.path.exists(csv_filename)

with open(csv_filename, 'a', newline='', encoding='utf-8') as csv_file:
    # 修改這裡：加入 'raw_question' 到欄位名稱中
    fieldnames = ['index', 'raw_question', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # 只在檔案不存在時寫入表頭
    if not file_exists:
        writer.writeheader()
        print(f"📁 CSV 檔案已建立: {csv_filename}")
    else:
        print(f"📁 CSV 檔案已存在，將追加資料: {csv_filename}")
    
    print(f"📊 開始處理區間 {START_INDEX}-{END_INDEX-1}\n")
    
    # 開始推論
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Gemini] 執行推論與評估"), START_INDEX):
        try:
            question = item["raw_question"]
            gt_answer = item["answer"]
            
            # 建立包含 one-shot 範例的完整對話
            chat = model.start_chat(history=[
                {"role": "user", "parts": [EXAMPLE_QUESTION]},
                {"role": "model", "parts": [f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"]}
            ])
            
            # 發送問題並取得回應
            response = chat.send_message(question)
            assistant_reply = response.text
            
            # 印出完整資訊
            print("\n" + "="*80)
            print(f"📝 樣本 #{idx} (絕對索引)")
            print("="*80)
            print(f"\n🤖 模型完整回答:")
            print("-"*80)
            print(assistant_reply)
            print("-"*80)
            
            # 記錄 ground truth
            all_ground_truths.append(gt_answer)
            
            # 計算 completion length
            completion_length = len(assistant_reply)
            print(f"\n📊 完成的回覆長度 (字元數): {completion_length}")
            all_completion_lengths.append(completion_length)
            
            # 提取預測的 ICD 編碼
            predicted_codes = extract_icd_answer(assistant_reply)
            print(f"🔍 提取的 ICD-10 編碼: {predicted_codes}")
            all_predictions.append(predicted_codes)
            
            print(f"\n🎯 Ground Truth Answer:")
            print(f"   {gt_answer}")
            
            # 計算 F1 分數
            f1_score = compute_f1(predicted_codes, gt_answer)
            print(f"✅ F1 分數: {f1_score:.4f}")
            all_f1_scores.append(f1_score)
            
            # 寫入 CSV (包含絕對索引)
            writer.writerow({
                'index': idx,
                'raw_question': question,  # 修改這裡：寫入原始問題
                'accuracy_reward': f1_score,    
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer if isinstance(gt_answer, str) else json.dumps(gt_answer), 
                'predictions_ans': json.dumps(predicted_codes) if isinstance(predicted_codes, list) else predicted_codes,
                'assistant_reply': assistant_reply if isinstance(assistant_reply, str) else json.dumps(assistant_reply)
            })
            csv_file.flush()
            
            print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
            print("="*80 + "\n")
            
        except Exception as e:
            print(f"\n❌ 處理第 {idx} 筆資料時發生錯誤:")
            print(f"   錯誤訊息: {e}")
            print(f"   跳過此筆資料繼續執行...\n")
            continue

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 7. 顯示最終統計 ---
if not all_predictions:
    print("⚠️  沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print(f"  Gemini-2.5-Flash 區間統計 [{START_INDEX}-{END_INDEX-1}]")
    print("="*50)
    print(f"✅ 總共處理: {len(all_f1_scores)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_f1_scores)/len(all_f1_scores):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} 字元")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已建立: gemini_flash_gt_reasonning_0_170.csv
📊 開始處理區間 0-169



[Gemini] 執行推論與評估:   1%|          | 1/170 [00:16<46:13, 16.41s/it]


📝 樣本 #0 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>Step-by-step reasoning:
1.  **D500 (Iron deficiency anemia secondary to blood loss (chronic))**: The medical record states "PMH significant for chronic anemia" and laboratory results show "H/H 7.8/25.4 (baseline Hct 31), iron studies with iron 49 ferritin 249," which are indicative of iron deficiency. The need for "1u pRBC transfusion" further supports the severity of anemia.
2.  **E538 (Deficiency of other specified B group vitamins)**: The patient was administered "FoLIC Acid 1 mg" in the ED, suggesting a deficiency in this B group vitamin.
3.  **H548 (Blindness, low vision of both eyes, unspecified)**: Although not explicitly stated as a chief complaint or acute finding, "recurrent falls" in an elderly patient can often be attributed to or exacerbated by visual impairment. Given the code is provided, it's inferred that this was part of the patient's documented medic

[Gemini] 執行推論與評估:   1%|          | 2/170 [00:41<59:32, 21.26s/it]


📝 樣本 #1 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>Step-by-step reasoning:
1.  **E118 (Type 2 diabetes mellitus with unspecified complications):** The patient's Past Medical History (PMHx) includes "DMII," which stands for Type 2 Diabetes Mellitus.
2.  **E669 (Obesity, unspecified):** The patient's PMHx lists "Obesity."
3.  **E785 (Hyperlipidemia, unspecified):** The patient's PMHx includes "HLD," which stands for Hyperlipidemia.
4.  **I071 (Tricuspid insufficiency, unspecified whether acute or chronic):** Although not explicitly stated in the provided text, tricuspid insufficiency is a common complication in patients with severe, chronic heart failure (such as ischemic cardiomyopathy with EF 25%) and pulmonary hypertension (often secondary to heart failure and COPD), which are present in this patient's history. Its presence is highly plausible given the overall cardiac status.
5.  **I129 (Hypertensive heart disease wi

...


📝 樣本 #167 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>Step-by-step reasoning:
1.  **Chief Complaint: "L jaw pain"**: This symptom is a strong indicator for several codes related to the jaw and bone.
    *   **C7951 (Secondary malignant neoplasm of bone):** Jaw pain, especially in an adult male, is highly suspicious for metastatic bone disease, often originating from a primary cancer.
    *   **M272 (Other specified disorders of jaw):** This code directly describes a disorder of the jaw, which would manifest as jaw pain. It's a specific anatomical diagnosis for the complaint.
    *   **M87180 (Osteonecrosis due to drugs, site unspecified):** Osteonecrosis of the jaw (ONJ) is a known complication in cancer patients, particularly those with bone metastases treated with bisphosphonates or denosumab. While drug use isn't mentioned, this code suggests a diagnosis of ONJ as the cause of the jaw pain.

2.  **Inference from C795

[Gemini] 執行推論與評估:  99%|█████████▉| 169/170 [1:01:37<00:19, 19.49s/it]


📝 樣本 #168 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>Step-by-step reasoning:
1.  **Chief Complaint: Fatigue.** Fatigue is a common, non-specific symptom that can be associated with various underlying conditions.
2.  **D599 (Acquired hemolytic anemia, unspecified):** Fatigue is a cardinal symptom of anemia. The presence of this code suggests that a diagnostic workup for the patient's fatigue revealed the presence of anemia, specifically an acquired hemolytic type.
3.  **B600 (Babesiosis):** Babesiosis is a parasitic infection that is a known cause of acquired hemolytic anemia. The assignment of this code alongside D599 strongly implies that further investigation into the patient's hemolytic anemia identified Babesiosis as the underlying etiology, thereby explaining both the anemia and the chief complaint of fatigue.
4.  **I10 (Essential (primary) hypertension):** This is a very common chronic medical condition in the ad

[Gemini] 執行推論與評估: 100%|██████████| 170/170 [1:02:01<00:00, 21.89s/it]


📝 樣本 #169 (絕對索引)

🤖 模型完整回答:
--------------------------------------------------------------------------------
<reasoning>Step-by-step reasoning:
1.  **C189 (Malignant neoplasm of colon, unspecified):** The patient has a history of "resected stage IIIB (T3 N2a) colon cancer" for which she is receiving adjuvant chemotherapy. This code specifies the primary malignancy.
2.  **G4733 (Obstructive sleep apnea (adult) (pediatric)):** This condition is not explicitly mentioned in the provided medical record. It is inferred as a known comorbidity or pre-existing condition for the patient.
3.  **G629 (Polyneuropathy, unspecified):** The patient is on "modified FOLFOX 6," which includes oxaliplatin. Oxaliplatin is well-known to cause peripheral neuropathy. This code likely represents the oxaliplatin-induced neuropathy, a common side effect of this chemotherapy regimen.
4.  **H3530 (Unspecified degenerative disease of retina):** This condition is not explicitly mentioned in the provided medical rec